<a href="https://colab.research.google.com/gist/karthikvenkataramani/9721129802e96dea373de8810b613ec0/designthinking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
!pip install langchain
!pip install langchain langchain_community
!pip install --upgrade langchain langchain_community
!pip install "langchain-community[groq]"
!pip install langchain-groq

from langchain import PromptTemplate, LLMChain
from langchain_groq import ChatGroq
from google.colab import userdata
import os
import json

# Groq API Key (from Google Colab secrets)
groq_api_key = userdata.get('GROQ_API_KEY')
if groq_api_key is None:
    raise ValueError("GROQ_API_KEY not found in Google Colab secrets. Please set it up in the Colab notebook settings.")

# Groq Model Details
groq_model_id = "llama3-8b-8192"

# Instantiate ChatGroq and assign to groq_llm
groq_llm = ChatGroq(model=groq_model_id, api_key=groq_api_key)

class DesignThinkingAgent:
    def __init__(self, name):
        self.name = name
        self.state = {}
        self.llm_tool = LLMChain(llm=groq_llm, prompt=PromptTemplate.from_template("{input}"))

    def run(self, input):
        return self.handle_message(input)

    def handle_message(self, message):
        prompt_text = f"You are {self.name}, a design thinking expert. Based on this input: '{message}'\n{self.get_stage_prompt()}"
        response = self.llm_tool.run(prompt_text)
        self.state["last_response"] = response
        return response

    def get_stage_prompt(self):
        if self.name == "Empathize Agent":
            return "Generate 3 user personas with their pain points related to the given input. Format as JSON array of objects with 'name' and 'pain_points' fields."
        elif self.name == "Define Agent":
            return "Formulate 3 clear problem statements based on the personas and pain points. Format as a JSON array of strings."
        elif self.name == "Ideate Agent":
            return "Generate 5 innovative ideas or use cases to address the problem statements. Format as a JSON array of strings."
        elif self.name == "Prototype Agent":
            return "Define a clear solution prototype based on the ideas. Provide a detailed description as a JSON object with 'name' and 'description' fields."
        elif self.name == "Test Agent":
            return "Evaluate the solution prototype. Provide a critical analysis of its strengths and weaknesses as a JSON object with 'strengths' and 'weaknesses' fields (both should be arrays of strings)."

def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Error parsing JSON. Raw output: {json_str}")
        return None

# Create agents
empathize_agent = DesignThinkingAgent("Empathize Agent")
define_agent = DesignThinkingAgent("Define Agent")
ideate_agent = DesignThinkingAgent("Ideate Agent")
prototype_agent = DesignThinkingAgent("Prototype Agent")
test_agent = DesignThinkingAgent("Test Agent")

# Main execution
print("Welcome to the Design Thinking AI Agent!")
vision = input("Please enter your vision or problem statement: ")

# Run the design thinking process
personas_raw = empathize_agent.run(vision)
personas = parse_json(personas_raw)
problem_statements_raw = define_agent.run(personas_raw if personas is None else json.dumps(personas))
problem_statements = parse_json(problem_statements_raw)
ideas_raw = ideate_agent.run(problem_statements_raw if problem_statements is None else json.dumps(problem_statements))
ideas = parse_json(ideas_raw)
prototype_raw = prototype_agent.run(ideas_raw if ideas is None else json.dumps(ideas))
prototype = parse_json(prototype_raw)
evaluation_raw = test_agent.run(prototype_raw if prototype is None else json.dumps(prototype))
evaluation = parse_json(evaluation_raw)

# Print results
print("\n--- Design Thinking Process Results ---")
print(f"Vision: {vision}")

print("\nPersonas and Pain Points:")
if personas:
    for persona in personas:
        print(f"- {persona['name']}: {persona['pain_points']}")
else:
    print("Raw output:", personas_raw)

print("\nProblem Statements:")
if problem_statements:
    for statement in problem_statements:
        print(f"- {statement}")
else:
    print("Raw output:", problem_statements_raw)

print("\nIdeas:")
if ideas:
    for idea in ideas:
        print(f"- {idea}")
else:
    print("Raw output:", ideas_raw)

print(f"\nSolution Prototype:")
if prototype:
    print(f"Name: {prototype.get('name', 'N/A')}")
    print(f"Description: {prototype.get('description', 'N/A')}")
else:
    print("Raw output:", prototype_raw)

print(f"\nEvaluation:")
if evaluation:
    print("Strengths:")
    for strength in evaluation.get('strengths', []):
        print(f"- {strength}")
    print("Weaknesses:")
    for weakness in evaluation.get('weaknesses', []):
        print(f"- {weakness}")
else:
    print("Raw output:", evaluation_raw)

# Continue the conversation if needed
while True:
    user_input = input("\nDo you want to refine any part of the process? (empathize/define/ideate/prototype/test/no): ").lower()
    if user_input == "no":
        break
    elif user_input in ["empathize", "define", "ideate", "prototype", "test"]:
        agent = globals()[f"{user_input}_agent"]
        result_raw = agent.run(vision)
        result = parse_json(result_raw)
        print(f"\nRefined {user_input.capitalize()} Stage Result:")
        if result:
            print(json.dumps(result, indent=2))
        else:
            print("Raw output:", result_raw)
    else:
        print("Invalid input. Please choose from empathize, define, ideate, prototype, test, or no.")

print("Thank you for using the Design Thinking AI Agent!")

Welcome to the Design Thinking AI Agent!
